# 量子位相推定 (Quantum Phase Estimation)
ユニタリ$U$と固有状態$\mid \psi \rangle$が準備されたとき、その固有値$\lambda = e^{-i\alpha}$の位相$\alpha$を求めることができる量子位相推定を確認しま\す。

$$
U \lvert \psi \rangle = e^{-i\alpha} \lvert \psi \rangle
$$

## 量子位相推定の量子回路
量子位相推定アルゴリズムの全体像は下記の通りです。

1. 固有ベクトルとしての量子状態の準備  
2. 固有値を量子状態として取り出す（位相キックバック）  
3. 固有値の量子状態をビットに変換（逆量子フーリエ変換）  

の３つからなります。


```
                         step2                   step3
      
|0> ----H----------------------*-------iQFT---
|0> ----H--------------*-------|-------iQFT---
|0> ----H--------*-----|-------|-------iQFT---
|0> ----H--*-----|-----|-------|-------iQFT---
                  |       |       |         |
                  |       |       |         |
|ψ> -------U1--U2---U4-- -U2n------------
step1
```

## 位相キックバック

位相キックバックについて復習しましょう。

まず、固有ベクトル$\lvert \psi \rangle$とそれに対応する固有値$e^{2\pi i \phi}$を持つユニタリについて、制御ユニタリ回路を導入します。  
これから確認するように、固有値は制御ビットの$\lvert 1 \rangle$状態の係数に埋め込むことができます。

ユニタリについての固有値方程式は以下のように書けます。

$$
U\lvert \psi \rangle = e^{2\pi i \phi} \lvert \psi \rangle
$$

量子状態$\lvert \psi \rangle$は準備されていると仮定すると、制御ユニタリ回路によって位相を取り出すことができます。

まず初期状態からアダマールゲートを使って結果を取り出したい方の量子ビットを重ね合わせ状態にします。

$$
\lvert 0 \rangle \lvert \psi \rangle \rightarrow \frac{\lvert 0\rangle + \lvert 1 \rangle}{\sqrt{2}} \lvert \psi \rangle = \frac{\lvert 0\rangle \lvert \psi \rangle + \lvert 1 \rangle \lvert \psi \rangle}{\sqrt{2}}
$$

次に制御ユニタリ回路を導入することで、制御量子ビットが$\lvert 1 \rangle$の状態にのみユニタリがかかります。

$$
\frac{\lvert 0\rangle \lvert \psi \rangle + \lvert 1 \rangle U \lvert \psi \rangle}{\sqrt{2}}
$$

固有値方程式より、次のように書けます。

$$
\frac{\lvert 0\rangle \lvert \psi \rangle + \lvert 1 \rangle e^{2\pi i \phi} \lvert \psi \rangle}{\sqrt{2}} = \frac{\lvert 0\rangle \lvert \psi \rangle +  e^{2\pi i \phi} \lvert 1 \rangle \lvert \psi \rangle}{\sqrt{2}} = \frac{\lvert 0\rangle +  e^{2\pi i \phi} \mid 1 \rangle}{\sqrt{2}} \lvert \psi \rangle
$$

以上より、固有値を係数として取り出すことが出来ます。

## 量子フーリエ変換

量子フーリエ変換についても復習します。

量子フーリエ変換はバイナリ(0 or 1)配列からなる入力を、その配列に対応する位相をもつ量子状態に変形することができます。量子フーリエ変換の逆回路である逆量子フーリエ変換を利用することで、上記の位相キックバックで移した位相をビット列で書き出すことができます。

$$
QFT:\lvert x \rangle \mapsto \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1} \omega_n^{xk}\lvert k\rangle
$$

<!-- 
$\omega_n = e^{\frac{2\pi i}{N}}$とすると、

$$
{F_N= 
\frac{1}{\sqrt{N}} 
\left[ 
 \begin{array}{rrrr} 
  1 & 1 & 1 & \cdots &1\\ 
  1 & \omega_n&\omega_n^2&\cdots&\omega_n^{N-1}\\ 
  1 & \omega_n^2&\omega_n^4&\cdots&\omega_n^{2(N-1)}\\ 
  1 & \omega_n^3&\omega_n^6&\cdots&\omega_n^{3(N-1)}\\ 
  \vdots&\vdots&\vdots&&\vdots\\ 
  1 & \omega_n^{N-1}&\omega_n^{2(N-1)}&\cdots&\omega_n^{(N-1)(N-1)} 
 \end{array} 
\right]
}
$$

のようになります。 -->

ビット列$x_n$...$x_1$を入力すると、出力される量子状態はそれに対応した位相を持ちます。

$$
QFT(\lvert x_n,x_{n-1},…,x_1 \rangle) = \frac{1}{\sqrt{N}}(\lvert 0 \rangle + e^{2\pi i [0.x_n]} \lvert 1 \rangle) \otimes … \otimes (\lvert 0 \rangle + e^{2\pi i [0.x_1x_2…x_n]} \lvert 1 \rangle)
$$

$$[0.x_1x_2…] = \frac{x_1}{2}+\frac{x_2}{2^2}+…$$

出力された量子状態のそれぞれの量子ビットの相対位相には、入力された状態(ビット列)が桁をずらしながらエンコードされています。ただし各量子ビットの$\lvert 1\rangle$の係数はすべて絶対値が$1/\sqrt{N}$なので、個別の量子ビットを測定しただけでは0と1が完全に50%ずつ出てしまうのが特徴です。

## 位相キックバック + 逆量子フーリエ変換

量子位相推定では、先程の位相キックバックを応用して次のような状態を準備します。

$$
\frac{1}{\sqrt{2^n}}\sum_{k=0}^{2^n-1} e^{i2\pi k\phi}\lvert k \rangle
$$

$\phi$ はユニタリの固有値で、これを求めたいのでした。  
このような状態を用意できれば、逆量子フーリエ変換によって、$\phi$ のバイナリエンコードビット列からなる量子状態 $\lvert \phi_n,\phi_{n-1},...,\phi_1\rangle$ へ変換することができます。  
よって最後に測定を行えば、$\phi$ を取り出すことができます。

各項でオリジナルの位相 $\phi$ に $k$ がかかっていますが、これは固有値を $k$ 回かけていることと同じです。よって $U^k$ のように同じユニタリ操作を $k$ 回実行します。すなわち、$k$ 回のControlled-Unitary操作を行えれば実現できます。

$$
\frac{\mid 0\rangle +  U^k \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle = \frac{\mid 0\rangle +  e^{2\pi i k \phi} \mid 1 \rangle}{\sqrt{2}} \mid \psi \rangle
$$

よって、各量子ビットに対して対応する $k$ 回の制御付きユニタリゲートをかけることで、上記の状態を準備できます。

## Zゲートの位相を推定する
例題をやってみます。まずはユニタリとしてZゲートを用意します。

$$
Z = \begin{pmatrix}
1&0\\
0&-1
\end{pmatrix}
$$

まずは手計算で答えを確認します。次の特性方程式を計算し、

$$
det\begin{pmatrix}
1-\lambda&0\\
0&-1-\lambda
\end{pmatrix} = 0
$$

固有値は $\lambda = 1,-1$ となります。  
固有ベクトルは、次のようになります。

$$
\begin{pmatrix}
1\\
0
\end{pmatrix},
\begin{pmatrix}
0\\
1
\end{pmatrix}
$$

早速確かめてみましょう。

## blueqatのインストール
pip経由でインストールを行います。

In [ ]:
!pip install blueqat

## 回路の全体概要
今回の回路の概要です。

```
|0> ----H--*--iQFT--M
                  |
|0> ------- Z--------
```

まず２量子ビットを準備し、それぞれ0番目、1番目とします。どちらも状態$\lvert 0 \rangle$からスタートします。

1. 1番目の量子ビットに固有状態を準備します。  
2. 0番目の量子ビットにアダマールゲートを作用させ、重ね合わせ状態とします。
3. 制御ユニタリとしてCZゲートをかけて、位相を0番目の量子ビットにキックバックします。  
4. 逆量子フーリエ変換を実行し、測定して位相を取り出します。

量子状態の準備について、まずは固有状態 $\lvert 0 \rangle$ について計算します。すなわち、初期状態のまま何もしません。

また1量子ビットの量子フーリエ変換はアダマールゲートと等価です。  
(アダマール行列はエルミートなので、この場合の逆量子フーリエ変換もアダマールゲートです)

よって、実装は以下のようになります。

In [3]:
from blueqat import Circuit

Circuit().h[0].cz[0,1].h[0].m[:].run(shots=100)

Counter({'00': 100})

よって $\phi = 0.0$ です。

求める固有値は、次のようになります。

$$
e^{2\pi i \phi} = e^{2\pi i \cdot 0} = e^0 = 1
$$

次に、用意する固有状態を $\lvert 1 \rangle$ にしてみます。

```
|0> --H--*--iQFT--M
               |
|0> --X--Z--------
```

In [4]:
Circuit().x[1].h[0].cz[0,1].h[0].m[:].run(shots=100)

Counter({'11': 100})

今度は $\phi = 1/2 = 0.5$ です。

よって固有値は次の通りです。

$$
e^{2\pi i \cdot 0.5} = -1
$$

## Xゲートの位相を推定する場合

Xゲートは、次のような行列です。

$$
X = 
\begin{pmatrix}
0&1\\
1&0
\end{pmatrix}
$$

固有ベクトルとしてまずは次を考えます。固有値は1です。

$$
\mid \psi \rangle = 
\begin{pmatrix}
1\\
1
\end{pmatrix}
$$

実行する回路は次の通りです。

```
|0> --H--*--H--M
         　　　　　　|
|0> --H--X-----
```

In [7]:
Circuit(2).h[:].cx[0,1].h[0].m[0].run(shots=100)

Counter({'00': 100})

こちらは、$\phi = 0.0$となり、

$$
\lambda = e^0=1
$$

となりました。

続いて固有ベクトルが次の場合を考えます。固有値は-1です。

$$
\mid \psi \rangle = 
\begin{pmatrix}
1\\
-1
\end{pmatrix}
$$


```
|0> --H---*--H--M
          　　　　　　|
|0> --HZ--X-----
```

In [5]:
Circuit(2).h[:].z[1].cx[0,1].h[0].m[0].run(shots=100)

Counter({'10': 100})

これより $\phi = 0.5$となり、

$$
\lambda = e^{2\pi i \cdot0.5}=-1
$$

となりました。